In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
train_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/train'
valid_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid'
test_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test'
# ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).
# in this example, we won't be doing any image augmentation
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
 

In [2]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers[:-5]:
    layer.trainable = False
last_layer = base_model.get_layer('global_average_pooling2d')

last_output = last_layer.output

x = Dense(10, activation='softmax', name='softmax')(last_output)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
import time
start_time = time.time()
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5', save_best_only=True)

history = new_model.fit_generator(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])
end_time = time.time()
train_time_in_minutes = (end_time - start_time) / 60.0
train_time = end_time - start_time
print(f"Training time: {train_time:.3f} seconds")
print(f"Training time in minutes: {train_time_in_minutes:.3f} minutes")

C:\Users\subeh\AppData\Local\Temp\ipykernel_18768\3998107238.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(train_batches, steps_per_epoch=18,


Epoch 1/20
18/18 [==============================] - 81s 4s/step - loss: 3.3542 - accuracy: 0.2791 - val_loss: 1.6612 - val_accuracy: 0.4556
Epoch 2/20
18/18 [==============================] - 77s 4s/step - loss: 1.4610 - accuracy: 0.4593 - val_loss: 1.1111 - val_accuracy: 0.5556
Epoch 3/20
18/18 [==============================] - 71s 4s/step - loss: 0.7877 - accuracy: 0.6944 - val_loss: 0.5143 - val_accuracy: 0.8444
Epoch 4/20
18/18 [==============================] - 72s 4s/step - loss: 0.4280 - accuracy: 0.8556 - val_loss: 0.2317 - val_accuracy: 0.9667
Epoch 5/20
18/18 [==============================] - 72s 4s/step - loss: 0.1640 - accuracy: 0.9500 - val_loss: 0.2679 - val_accuracy: 0.9444
Epoch 6/20
18/18 [==============================] - 74s 4s/step - loss: 0.1983 - accuracy: 0.9333 - val_loss: 0.3023 - val_accuracy: 0.8889
Epoch 7/20
18/18 [==============================] - 73s 4s/step - loss: 0.1790 - accuracy: 0.9611 - val_loss: 0.1695 - val_accuracy: 0.9889
Epoch 8/20
18/18 [==

In [4]:
  def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test')
valid_files, valid_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid')

from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image #wrire this to solve the error 'image.load_img'
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
val_tensors = preprocess_input(paths_to_tensor(valid_files))

new_model.load_weights('signlanguage.model.hdf5')
print('\nval loss: {:.4f}\nval accuracy: {:.4f}'.format(*new_model.evaluate(val_tensors, valid_targets)))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 485.89it/s]


10/10 [==============================] - 69s 7s/step - loss: 0.2078 - accuracy: 0.9400

val loss: 0.2078
val accuracy: 0.9400
2/2 [==============================] - 12s 4s/step - loss: 0.1917 - accuracy: 0.9200

Testing loss: 0.1917
Testing accuracy: 0.9200


In [6]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [9]:
import random
import cv2
import os
# Define the data augmentation functions
def random_crop(image):
    # Randomly crop a portion of the image
    (h, w) = image.shape[:2]
    crop_h, crop_w = int(h * 0.8), int(w * 0.8)
    top = random.randint(0, h - crop_h)
    left = random.randint(0, w - crop_w)
    bottom = top + crop_h
    right = left + crop_w
    cropped = image[top:bottom, left:right]
    return cropped

def random_zoom(image):
    # Randomly zoom in or out on the image
    (h, w) = image.shape[:2]
    zoom_scale = random.uniform(1.0, 1.5)
    new_h, new_w = int(h * zoom_scale), int(w * zoom_scale)
    top = random.randint(0, new_h - h)
    left = random.randint(0, new_w - w)
    bottom = top + h
    right = left + w
    zoomed = cv2.resize(image[top:bottom, left:right], (w, h))
    return zoomed

def adjust_brightness(image):
    # Adjust the brightness of the image
    alpha = random.uniform(0.5, 1.5)
    beta = random.randint(-50, 50)
    adjusted = cv2.addWeighted(image, alpha, image, 0, beta)
    return adjusted

def adjust_contrast(image):
    # Adjust the contrast of the image
    alpha = random.uniform(0.5, 1.5)
    mean = cv2.mean(image)[0]
    adjusted = cv2.addWeighted(image, alpha, image, 0, mean * (1 - alpha))
    return adjusted

def random_color_jitter(image):
    # Randomly adjust the hue, saturation, and brightness of the image
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_shift = random.randint(0, 30)
    hsv[:, :, 0] = hsv[:, :, 0] + hue_shift
    saturation_scale = random.uniform(0.5, 1.5)
    hsv[:, :, 1] = hsv[:, :, 1] * saturation_scale
    brightness_scale = random.uniform(0.5, 1.5)
    hsv[:, :, 2] = hsv[:, :, 2] * brightness_scale
    jittered = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return jittered

def add_gaussian_noise(image):
    # Add Gaussian noise to the image
    mean = random.randint(0, 20)
    std_dev = random.uniform(0.0, 10.0)
    noise = image.copy()
    cv2.randn(noise, mean, std_dev)
    noisy = cv2.add(image, noise)
    return noisy

def apply_blur(image):
    # Apply a blur filter to the image
    k_size = random.choice([3, 5, 7])
    blurred = cv2.GaussianBlur(image, (k_size, k_size), 0)
    return blurred

def rotate_image(image):
    # Rotate the image by 45 degrees
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, 45, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def flip_image(image):
    # Flip the image horizontally
    flipped = cv2.flip(image, 1)
    return flipped

    # Loop through each folder
for i in range(10):
    folder_name = str(i)
    print(f"Applying data augmentation to images in folder {folder_name}")
    
    # Get the path to the folder
    folder_path = os.path.join("C:/Users/subeh/OneDrive/Desktop/deep_learnin/chapter_06/sign_language_project/dataset/train", folder_name)

    # Loop through each image in the folder
    for filename in os.listdir(folder_path):
        # Load the image
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        cropped_img = random_crop(img)
        zoomed_img = random_zoom(img)
        adjusted_img = adjust_brightness(img)
        jittered_img = random_color_jitter(img)
        noisy_img = add_gaussian_noise(img)
        blurred_img = apply_blur(img)
        rotated_img = rotate_image(img)
        flipped_img = flip_image(img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_cropped.jpg"), cropped_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_zoomed.jpg"), zoomed_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_adjusted.jpg"), adjusted_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_jittered.jpg"), jittered_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_noisy.jpg"), noisy_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_blurred.jpg"), blurred_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_rotated.jpg"), rotated_img)
        cv2.imwrite(os.path.join(folder_path, f"{filename.split('.')[0]}_flipped.jpg"), flipped_img)

Applying data augmentation to images in folder 0
Applying data augmentation to images in folder 1
Applying data augmentation to images in folder 2
Applying data augmentation to images in folder 3
Applying data augmentation to images in folder 4
Applying data augmentation to images in folder 5
Applying data augmentation to images in folder 6
Applying data augmentation to images in folder 7
Applying data augmentation to images in folder 8
Applying data augmentation to images in folder 9


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
train_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/train'
valid_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/valid'
test_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/test'
# ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).
# in this example, we won't be doing any image augmentation
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Found 15408 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    


In [3]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers[:-5]:
    layer.trainable = False
last_layer = base_model.get_layer('global_average_pooling2d')

last_output = last_layer.output

x = Dense(10, activation='softmax', name='softmax')(last_output)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
import time
start_time = time.time()
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5', save_best_only=True)

history = new_model.fit_generator(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])
end_time = time.time()
train_time_in_minutes = (end_time - start_time) / 60.0
train_time = end_time - start_time
print(f"Training time: {train_time:.3f} seconds")
print(f"Training time in minutes: {train_time_in_minutes:.  } minutes")

C:\Users\subeh\AppData\Local\Temp\ipykernel_23476\3998107238.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(train_batches, steps_per_epoch=18,


Epoch 1/20
18/18 [==============================] - 74s 4s/step - loss: 3.2871 - accuracy: 0.1333 - val_loss: 2.0747 - val_accuracy: 0.1556
Epoch 2/20
18/18 [==============================] - 73s 4s/step - loss: 2.0650 - accuracy: 0.1944 - val_loss: 1.6372 - val_accuracy: 0.5000
Epoch 3/20
18/18 [==============================] - 76s 4s/step - loss: 1.5648 - accuracy: 0.5000 - val_loss: 1.0870 - val_accuracy: 0.6444
Epoch 4/20
18/18 [==============================] - 73s 4s/step - loss: 1.1051 - accuracy: 0.6056 - val_loss: 0.7069 - val_accuracy: 0.7778
Epoch 5/20
18/18 [==============================] - 72s 4s/step - loss: 0.8900 - accuracy: 0.6722 - val_loss: 0.3095 - val_accuracy: 0.9000
Epoch 6/20
18/18 [==============================] - 70s 4s/step - loss: 0.7143 - accuracy: 0.7833 - val_loss: 0.8361 - val_accuracy: 0.6667
Epoch 7/20
18/18 [==============================] - 75s 4s/step - loss: 0.7026 - accuracy: 0.7611 - val_loss: 0.7527 - val_accuracy: 0.7444
Epoch 8/20
18/18 [==

In [4]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test')
valid_files, valid_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid')

from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image #wrire this to solve the error 'image.load_img'
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
val_tensors = preprocess_input(paths_to_tensor(valid_files))

new_model.load_weights('signlanguage.model.hdf5')
print('\nval loss: {:.4f}\nval accuracy: {:.4f}'.format(*new_model.evaluate(val_tensors, valid_targets)))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 510.68it/s]


10/10 [==============================] - 69s 7s/step - loss: 0.3015 - accuracy: 0.9067

val loss: 0.3015
val accuracy: 0.9067
2/2 [==============================] - 12s 4s/step - loss: 0.2620 - accuracy: 0.8800

Testing loss: 0.2620
Testing accuracy: 0.8800


In [5]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
train_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/train'
valid_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/valid'
test_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/test'
# ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).
# in this example, we won't be doing any image augmentation
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Found 15408 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    


In [7]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers[:-5]:
    layer.trainable = False
last_layer = base_model.get_layer('global_average_pooling2d_1')

last_output = last_layer.output

x = Dense(10, activation='softmax', name='softmax')(last_output)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
import time
start_time = time.time()
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5', save_best_only=True)

history = new_model.fit_generator(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])


Epoch 1/20


C:\Users\subeh\AppData\Local\Temp\ipykernel_23476\13107789.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(train_batches, steps_per_epoch=18,


18/18 [==============================] - 79s 4s/step - loss: 3.4729 - accuracy: 0.1278 - val_loss: 2.2685 - val_accuracy: 0.1000
Epoch 2/20
18/18 [==============================] - 72s 4s/step - loss: 2.2208 - accuracy: 0.1167 - val_loss: 2.1704 - val_accuracy: 0.2667
Epoch 3/20
18/18 [==============================] - 72s 4s/step - loss: 2.0416 - accuracy: 0.2556 - val_loss: 1.8336 - val_accuracy: 0.3778
Epoch 4/20
18/18 [==============================] - 72s 4s/step - loss: 1.8876 - accuracy: 0.3833 - val_loss: 1.4131 - val_accuracy: 0.6111
Epoch 5/20
18/18 [==============================] - 74s 4s/step - loss: 1.6845 - accuracy: 0.4222 - val_loss: 1.3071 - val_accuracy: 0.5333
Epoch 6/20
18/18 [==============================] - 72s 4s/step - loss: 1.3564 - accuracy: 0.5556 - val_loss: 1.0786 - val_accuracy: 0.6222
Epoch 7/20
18/18 [==============================] - 72s 4s/step - loss: 1.3054 - accuracy: 0.5944 - val_loss: 0.7459 - val_accuracy: 0.8111
Epoch 8/20
18/18 [=============

ValueError: Format specifier missing precision

In [9]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test')
valid_files, valid_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid')

from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image #wrire this to solve the error 'image.load_img'
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
val_tensors = preprocess_input(paths_to_tensor(valid_files))

new_model.load_weights('signlanguage.model.hdf5')
print('\nval loss: {:.4f}\nval accuracy: {:.4f}'.format(*new_model.evaluate(val_tensors, valid_targets)))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets))) 

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 478.99it/s]


10/10 [==============================] - 69s 7s/step - loss: 0.2292 - accuracy: 0.9500

val loss: 0.2292
val accuracy: 0.9500
2/2 [==============================] - 12s 4s/step - loss: 0.1409 - accuracy: 0.9800

Testing loss: 0.1409
Testing accuracy: 0.9800


In [1]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
train_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/train'
valid_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid'
test_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test'
# ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).
# in this example, we won't be doing any image augmentation
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
 

In [3]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers[:-13]:
    layer.trainable = False
last_layer = base_model.get_layer('global_average_pooling2d')

last_output = last_layer.output

x = Dense(10, activation='softmax', name='softmax')(last_output)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
import time
start_time = time.time()
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5', save_best_only=True)

history = new_model.fit_generator(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])
end_time = time.time()
train_time_in_minutes = (end_time - start_time) / 60.0
train_time = end_time - start_time
print(f"Training time: {train_time:.3f} seconds")
print(f"Training time in minutes: {train_time_in_minutes:.3f} minutes")

C:\Users\subeh\AppData\Local\Temp\ipykernel_17512\455968399.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(train_batches, steps_per_epoch=18,


Epoch 1/20
18/18 [==============================] - 136s 8s/step - loss: 3.2281 - accuracy: 0.1333 - val_loss: 2.2986 - val_accuracy: 0.1000
Epoch 2/20
18/18 [==============================] - 160s 9s/step - loss: 2.2961 - accuracy: 0.1500 - val_loss: 2.3013 - val_accuracy: 0.0667
Epoch 3/20
18/18 [==============================] - 128s 7s/step - loss: 2.3134 - accuracy: 0.0889 - val_loss: 2.3032 - val_accuracy: 0.1000
Epoch 4/20
18/18 [==============================] - 117s 7s/step - loss: 2.3038 - accuracy: 0.0944 - val_loss: 2.3022 - val_accuracy: 0.1111
Epoch 5/20
18/18 [==============================] - 119s 7s/step - loss: 2.3023 - accuracy: 0.1333 - val_loss: 2.3018 - val_accuracy: 0.1333
Epoch 6/20
18/18 [==============================] - 113s 6s/step - loss: 2.3019 - accuracy: 0.1611 - val_loss: 2.3017 - val_accuracy: 0.1333
Epoch 7/20
18/18 [==============================] - 114s 6s/step - loss: 2.3015 - accuracy: 0.1512 - val_loss: 2.3014 - val_accuracy: 0.1222
Epoch 8/20
18

In [4]:
  def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test')
valid_files, valid_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid')

from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image #wrire this to solve the error 'image.load_img'
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
val_tensors = preprocess_input(paths_to_tensor(valid_files))

new_model.load_weights('signlanguage.model.hdf5')
print('\nval loss: {:.4f}\nval accuracy: {:.4f}'.format(*new_model.evaluate(val_tensors, valid_targets)))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 695.72it/s]


10/10 [==============================] - 61s 6s/step - loss: 2.2078 - accuracy: 0.1800

val loss: 2.2078
val accuracy: 0.1800
2/2 [==============================] - 12s 4s/step - loss: 2.1868 - accuracy: 0.1800

Testing loss: 2.1868
Testing accuracy: 0.1800


In [5]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
train_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/train'
valid_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/valid'
test_path  = 'C:/Users/subeh/OneDrive/Desktop/deep_learning/chapter_06/sign_language_project/dataset/test'
# ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).
# in this example, we won't be doing any image augmentation
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224,224), 
                                                         batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                         target_size=(224,224), 
                                                         batch_size=30)

test_batches = ImageDataGenerator().flow_from_directory(test_path, 
                                                        target_size=(224,224), 
                                                        batch_size=50, 
                                                        shuffle=False)
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3), pooling='avg')
base_model.summary()

Found 15408 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    


In [7]:
# iterate through its layers and lock them to make them not trainable with this code
for layer in base_model.layers[:-13]:
    layer.trainable = False
last_layer = base_model.get_layer('global_average_pooling2d_1')

last_output = last_layer.output

x = Dense(10, activation='softmax', name='softmax')(last_output)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
import time
start_time = time.time()
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5', save_best_only=True)

history = new_model.fit_generator(train_batches, steps_per_epoch=18,
                   validation_data=valid_batches, validation_steps=3, epochs=20, verbose=1, callbacks=[checkpointer])

C:\Users\subeh\AppData\Local\Temp\ipykernel_17512\3531255438.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = new_model.fit_generator(train_batches, steps_per_epoch=18,


Epoch 1/20
18/18 [==============================] - 112s 6s/step - loss: 3.1225 - accuracy: 0.0449 - val_loss: 2.2897 - val_accuracy: 0.1333
Epoch 2/20
18/18 [==============================] - 106s 6s/step - loss: 2.3054 - accuracy: 0.0889 - val_loss: 2.3016 - val_accuracy: 0.1111
Epoch 3/20
18/18 [==============================] - 105s 6s/step - loss: 2.3020 - accuracy: 0.1222 - val_loss: 2.3025 - val_accuracy: 0.1222
Epoch 4/20
18/18 [==============================] - 107s 6s/step - loss: 2.3047 - accuracy: 0.1278 - val_loss: 2.3162 - val_accuracy: 0.0556
Epoch 5/20
18/18 [==============================] - 106s 6s/step - loss: 2.3043 - accuracy: 0.1222 - val_loss: 2.3028 - val_accuracy: 0.0778
Epoch 6/20
18/18 [==============================] - 110s 6s/step - loss: 2.3028 - accuracy: 0.1167 - val_loss: 2.3026 - val_accuracy: 0.1111
Epoch 7/20
18/18 [==============================] - 114s 6s/step - loss: 2.3026 - accuracy: 0.1111 - val_loss: 2.3025 - val_accuracy: 0.0889
Epoch 8/20
18

In [8]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/test')
valid_files, valid_targets = load_dataset('C:/Users/subeh/OneDrive/Desktop/deep_learning_for_vision_systems/chapter_06/sign_language_project/dataset/valid')

from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image #wrire this to solve the error 'image.load_img'
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))
val_tensors = preprocess_input(paths_to_tensor(valid_files))

new_model.load_weights('signlanguage.model.hdf5')
print('\nval loss: {:.4f}\nval accuracy: {:.4f}'.format(*new_model.evaluate(val_tensors, valid_targets)))
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets))) 

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 643.00it/s]


10/10 [==============================] - 59s 6s/step - loss: 2.3132 - accuracy: 0.0733

val loss: 2.3132
val accuracy: 0.0733
2/2 [==============================] - 10s 3s/step - loss: 2.3116 - accuracy: 0.0800

Testing loss: 2.3116
Testing accuracy: 0.0800
